## 1. Obtención de datos

Conéctense a la API de Banxico para descargar la serie histórica del tipo de cambio FIX.

Usen al menos 2 años de datos para entrenar.

Conserven los últimos 15 días como conjunto de prueba (test set).

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from datetime import datetime

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

from scipy.stats import boxcox
from scipy.special import inv_boxcox

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [4]:
import requests
import pandas as pd

# Tu token de Banxico
token = "b2b8aef3c6559490348dbc3e9bfbe878054a181310b39ac2e33793ac6186b216"

# Serie del tipo de cambio FIX (peso/dólar)
serie_id = "SF43718"

# Endpoint
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"

# Headers con el token
headers = {"Bmx-Token": token}

# Llamada a la API
response = requests.get(url, headers=headers)
data = response.json()

# Extraer datos
datos = data["bmx"]["series"][0]["datos"]   

# Convertir a DataFrame
df = pd.DataFrame(datos)
df["Fecha"] = pd.to_datetime(df["fecha"], dayfirst=True)
df["TipoCambio"] = pd.to_numeric(df["dato"], errors="coerce")
df.set_index('Fecha', inplace=True) 
df.drop(columns=['fecha', 'dato'], inplace= True)
df.head()

,TipoCambio
Fecha,
1991-11-12,3.0735
1991-11-13,3.0712
1991-11-14,3.0718
1991-11-15,3.0684
1991-11-18,3.0673


In [5]:
print(df.index.dtype)       
print(df.shape)
print(df.head(3))
print(df.tail(3))

datetime64[ns]
(8520, 1)
            TipoCambio
Fecha                 
1991-11-12      3.0735
1991-11-13      3.0712
1991-11-14      3.0718
            TipoCambio
Fecha                 
2025-10-01     18.3477
2025-10-02     18.4843
2025-10-03     18.3902


In [6]:
import plotly.io as pio
pio.renderers.default = "browser"

fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

fig.show()

In [7]:
fecha_inicio = datetime(2021, 1, 1)
hoy = datetime.today()

dias = (hoy - fecha_inicio).days
print(f"Días entre 2021-01-01 y hoy: {dias}")

# Asegúrate de que el índice sea datetime
df.index = pd.to_datetime(df.index)

# Filtrar desde 2021-01-01 hasta hoy
fecha_inicio = "2021-01-01"
hoy = pd.Timestamp.today()

df = df.loc[fecha_inicio:hoy]

# Graficar con Plotly
fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

# Mostrar gráfico interactivo
fig.show()

Días entre 2021-01-01 y hoy: 1736


In [8]:
from sklearn.preprocessing import MinMaxScaler
# Normalizar datos
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["TipoCambio"]])

# Función para crear ventanas
def create_dataset(series, window=15):
    X, y = [], []
    for i in range(len(series)-window):
        X.append(series[i:(i+window), 0])
        y.append(series[i+window, 0])
    return np.array(X), np.array(y)

window_size = 15
X, y = create_dataset(df_scaled, window_size)

# Separar en train y test (últimos 15 días)

n_test = 15
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]

fechas_test = df.index[window_size:][-n_test:]

In [9]:
display(X_train)
display(X_test)
display(y_train)
display(y_test)

array([[0.64018385, 0.65805793, 0.61816955, ..., 0.59159554, 0.6207777 ,
        0.66528051],
       [0.65805793, 0.61816955, 0.65191143, ..., 0.6207777 , 0.66528051,
        0.69747209],
       [0.61816955, 0.65191143, 0.66294594, ..., 0.66528051, 0.69747209,
        0.67109871],
       ...,
       [0.44913183, 0.4416539 , 0.44466331, ..., 0.44174509, 0.42742759,
        0.42301379],
       [0.4416539 , 0.44466331, 0.41207047, ..., 0.42742759, 0.42301379,
        0.41942073],
       [0.44466331, 0.41207047, 0.41969432, ..., 0.42301379, 0.41942073,
        0.41152331]], shape=(1168, 15))

array([[0.41207047, 0.41969432, 0.42835777, 0.42961625, 0.4225031 ,
        0.42100751, 0.42137229, 0.43253447, 0.42890494, 0.44174509,
        0.42742759, 0.42301379, 0.41942073, 0.41152331, 0.39997811],
       [0.41969432, 0.42835777, 0.42961625, 0.4225031 , 0.42100751,
        0.42137229, 0.43253447, 0.42890494, 0.44174509, 0.42742759,
        0.42301379, 0.41942073, 0.41152331, 0.39997811, 0.39031152],
       [0.42835777, 0.42961625, 0.4225031 , 0.42100751, 0.42137229,
        0.43253447, 0.42890494, 0.44174509, 0.42742759, 0.42301379,
        0.41942073, 0.41152331, 0.39997811, 0.39031152, 0.36984752],
       [0.42961625, 0.4225031 , 0.42100751, 0.42137229, 0.43253447,
        0.42890494, 0.44174509, 0.42742759, 0.42301379, 0.41942073,
        0.41152331, 0.39997811, 0.39031152, 0.36984752, 0.36295324],
       [0.4225031 , 0.42100751, 0.42137229, 0.43253447, 0.42890494,
        0.44174509, 0.42742759, 0.42301379, 0.41942073, 0.41152331,
        0.39997811, 0.39031152, 0.36984752, 

array([0.69747209, 0.67109871, 0.70786824, ..., 0.41942073, 0.41152331,
       0.39997811], shape=(1168,))

array([0.39031152, 0.36984752, 0.36295324, 0.36939155, 0.37453491,
       0.37800029, 0.36249726, 0.38219523, 0.38483986, 0.37331291,
       0.36751295, 0.36450354, 0.36696578, 0.39188006, 0.3747173 ])

## 2. Preparación de datos

Normalicen los datos (ejemplo: MinMaxScaler).

Construyan ventanas deslizantes de tamaño definido por ustedes (ejemplo: 10, 15, 30 días).

Justifiquen el tamaño de la ventana: ¿por qué ese número de rezagos es razonable?
R = Dado que vamos a predecir 15 días, probamos un aprendizaje de 15 días por 15 días. 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop

# ==============================
# PARÁMETROS (ajustables rápido)
# ==============================
WINDOW_SIZE = 10        # Número de columnas de ventana deslizante (len)
HIDDEN_LAYERS = [64, 32]  # Neuronas por capa oculta (puedes agregar/quitar capas)
ACTIVATION = "relu"     # Función de activación (relu, tanh, sigmoid, etc.)
OUTPUT_ACTIVATION = "linear"  # Salida para regresión
OPTIMIZER = Adam(learning_rate=0.001)  # Optimizador
EPOCHS = 50             # Número de épocas
BATCH_SIZE = 16         # Tamaño del batch
LOSS = "mse"            # Error cuadrático medio para regresión
METRICS = ["mae"]       # Métrica adicional (error absoluto medio)

# ==============================
# FUNCIONES PRINCIPALES
# ==============================

def preparar_datos(df, window_size):
    """
    Separa X e y del DataFrame.
    Asume que las últimas columnas después de 'fecha' y 'dolar' son la ventana deslizante.
    """
    X = df.iloc[:, -window_size:].values  # Últimas columnas = ventana
    y = df["dolar"].values                # Columna objetivo
    return X, y

def construir_modelo(input_dim, hidden_layers, activation, output_activation, optimizer, loss, metrics):
    """
    Construcción de la red neuronal feed-forward (FFNN).
    """
    model = Sequential()
    
    # Capa de entrada + primera oculta
    model.add(Dense(hidden_layers[0], activation=activation, input_dim=input_dim))
    
    # Capas ocultas adicionales
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation=activation))
    
    # Capa de salida (regresión → 1 neurona)
    model.add(Dense(1, activation=output_activation))
    
    # Compilar
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

def entrenar_modelo(model, X, y, epochs, batch_size):
    """
    Entrena el modelo con los datos dados.
    """
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)
    return history

# ==============================
# MAIN
# ==============================

def main(df):
    # Preparar datos
    X, y = preparar_datos(df, WINDOW_SIZE)
    
    # Construir modelo
    model = construir_modelo(
        input_dim=X.shape[1],
        hidden_layers=HIDDEN_LAYERS,
        activation=ACTIVATION,
        output_activation=OUTPUT_ACTIVATION,
        optimizer=OPTIMIZER,
        loss=LOSS,
        metrics=METRICS
    )
    
    # Entrenar modelo
    history = entrenar_modelo(model, X, y, EPOCHS, BATCH_SIZE)
    
    return model, history



# mari

# remi

# esteban